In [1]:
import torch
import torchvision.datasets as datasets
import os
import foolbox
import torchvision.models as models
import numpy as np
import cv2
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

BATCH_SIZE = 64
datapath = '/home/user/datasets/ImageNet/'
traindir = os.path.join(datapath, 'train')
labeldir = '/home/user/datasets/ImageNet/class_to_idx.txt'


train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
#         transforms.Resize(224),
       transforms.CenterCrop(224),
        transforms.ToTensor(),
   #      transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                 	     std=[0.229, 0.224, 0.225])
    ])
)


# train_loader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=BATCH_SIZE, shuffle=False,
#     num_workers=1, pin_memory=True, sampler=None)

In [2]:
resnet101 = models.resnet101(pretrained=True).eval()
if torch.cuda.is_available():
    resnet101 = resnet101.cuda()
else:
    print('===============')
mean = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
std = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))
fmodel = foolbox.models.PyTorchModel(
    resnet101, bounds=(0, 1), num_classes=1000, preprocessing=(mean, std))

In [3]:
from scipy import ndimage
import tensorflow as tf
from abdm.abdm import ABDM
transform = transforms.Compose([transforms.ToTensor()])

In [4]:
img_id=[]        #ori images ID list
img_ori=[]       #ori images list
img_adv=[]       #adv images list
img_label=[]     #ori images labels list
abdm_image=[]     #abdm images list  

wrong_oriimg=0
right_advimg=0
wrong_advimg=0
right_abdmimg=0
wrong_abdmimg=0
for num in range(1194000,1194100):
    image, target = train_dataset[num]
    image= np.array(image)
    print('predicted class', np.argmax(fmodel.predictions(image)),', ground truth class',target)
    tempclass1=str(np.argmax(fmodel.predictions(image)))
    tempclass2=str(target)
    if(tempclass1!=tempclass2):
        wrong_oriimg=wrong_oriimg+1
        continue
        
    #dp_attack = foolbox.attacks.FGSM(fmodel)
    dp_attack = foolbox.attacks.deepfool.DeepFoolAttack(fmodel, distance=foolbox.distances.Linfinity)
    #dp_attack = foolbox.attacks.PGD(fmodel, distance=foolbox.distances.Linfinity)
    adversarial = dp_attack(image, target)
    try:
        print('adversarial class', np.argmax(fmodel.predictions(adversarial)))
    except: 
        wrong_advimg=wrong_advimg+1
        print('error')
        continue
    else:
        right_advimg=right_advimg+1
        print('adversarial class', np.argmax(fmodel.predictions(adversarial)))

    
    #===============abdm start (0.0)=========================================
    im=adversarial
    im = transform(im).numpy()
    im = transform(im).numpy()
    image_show=im
    #im=im.resize(3,224,224)
    print('ori image shape is :',im.shape)
    print("===========================================================")
    im = im.reshape(1, 224, 224, 3)
    im = im.astype('float32')
    #print('img-over')
    out_size = (224, 224)
    batch = np.append(im, im, axis=0)
    batch = np.append(batch, im, axis=0)
    num_batch = 3 
    x = tf.placeholder(tf.float32, [None, 224, 224, 3])
    x = tf.cast(batch, 'float32')
    print('begin---')
    with tf.variable_scope('spatial_transformer_0'):
        n_fc = 6
        w_fc1 = tf.Variable(tf.Variable(tf.zeros([224 * 224 * 3, n_fc]), name='W_fc1'))
        initial = np.array([[0.5, 0, 0], [0, 0.5, 0]])
        initial = initial.astype('float32')
        initial = initial.flatten()  
        b_fc1 = tf.Variable(initial_value=initial, name='b_fc1')   
        h_fc1 = tf.matmul(tf.zeros([num_batch, 224 * 224 * 3]), w_fc1) + b_fc1
        print(x, h_fc1, out_size)
        h_trans = ABDM(x, h_fc1, out_size)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    y = sess.run(h_trans, feed_dict={x: batch})
    abdmimg_temp=transform(y[0]).numpy()
    
    adv_class=str(np.argmax(fmodel.predictions(abdmimg_temp)))
    orilabel=str(target)
    print('adversarial class', np.argmax(fmodel.predictions(abdmimg_temp)))
    print('ori class', orilabel)
    if(adv_class==orilabel):
        # put images and labels into list
        img_ori.append(image)
        img_adv.append(adversarial)
        img_label.append(target)
        img_id.append(num)
        abdm_image.append(abdmimg_temp)
        print(len(img_id))
        right_abdmimg=right_abdmimg+1
    else:
        print('can not use this img')
        wrong_abdmimg=wrong_abdmimg+1
        continue

        
ori_right=(100-wrong_oriimg)/100
adv_right=(wrong_oriimg+wrong_advimg)/100
abdm_right=right_abdmimg/100
abdm_right2=right_abdmimg/(right_abdmimg+wrong_abdmimg)

print('clean image accuracy:  %.2f%%' % (ori_right * 100))
print('adv image accuracy:  %.2f%%' % (adv_right * 100))
print('abdm image accuracy:  %.2f%%' % (abdm_right * 100 ))      
print('abdm image accuracy:  %.2f%%' % (abdm_right2 * 100 ))   

predicted class 933 , ground truth class 933
adversarial class 941
adversarial class 941
ori image shape is : (224, 224, 3)
begin---
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Tensor("Cast/x:0", shape=(3, 224, 224, 3), dtype=float32) Tensor("spatial_transformer_0/add:0", shape=(3, 6), dtype=float32) (224, 224)
adversarial class 933
ori class 933
1
predicted class 933 , ground truth class 933
adversarial class 934
adversarial class 934
ori image shape is : (224, 224, 3)
begin---
Tensor("Cast_1/x:0", shape=(3, 224, 224, 3), dtype=float32) Tensor("spatial_transformer_0_1/add:0", shape=(3, 6), dtype=float32) (224, 224)
adversarial class 933
ori class 933
2
predicted class 933 , ground truth class 933
adversarial class 121
adversarial class 121
ori image shape is : (224, 224, 3)
begin---
Tensor("Cast_2/x:0", shape=(3, 224, 224, 3), dtype=float32) Tensor("spatial_transformer_0_2/add:0", shape=(3, 6), dtype=float32) (224, 224)
adversarial class 933
ori class 933
3
predicted class 933 , ground truth class 933
adversarial class 960
adversarial class 960
ori image shape is : (224, 224, 3)
begin---
Tensor("Cast_3/x:0", shape=(3, 224, 224, 3), dtype=float32) Tensor("spa

KeyboardInterrupt: 